In [1]:
import pandas as pd
import numpy as np
from data import * 
from expenses import * 
from revenues import * 
from other import *
from model import *


from enum import Enum

# calculation 

In [2]:
### base case

In [3]:
# base case
_, _, roe, _ = run_model(ppa_df, exp_df, debt_service_df, dep_df)
roe

24.890409338211366

### exchange rate fluctiations affecting EPC contract price 

In [4]:
# TODO add to inputs! 

current_exchange = 1.30 # currently = 1.30 eur => us 

# price has fluctuated from 0.95 to 1.39 over past 10 years 
# https://www.xe.com/currencycharts/?from=EUR&to=USD&view=10Y
# lets do the full range, and then some, +/- 0.15
currs = np.round(np.arange(0.80, 1.55, 0.05),2)

# 0.01 increase in exchange rate => EPC contract price increases / decreases by 1,670,000
epc_fluct = 1670e3/0.01

devs = np.round(currs - current_exchange,3)
dev_cash = np.round(devs * epc_fluct,3)

print(currs, devs, dev_cash)

[0.8  0.85 0.9  0.95 1.   1.05 1.1  1.15 1.2  1.25 1.3  1.35 1.4  1.45
 1.5 ] [-0.5  -0.45 -0.4  -0.35 -0.3  -0.25 -0.2  -0.15 -0.1  -0.05  0.    0.05
  0.1   0.15  0.2 ] [-83500000. -75150000. -66800000. -58450000. -50100000. -41750000.
 -33400000. -25050000. -16700000.  -8350000.         0.   8350000.
  16700000.  25050000.  33400000.]


In [5]:
def eval_epc_fluct(value, const_exp=const_exp, op_exp=op_exp, corp_exp=corp_exp, const_df=const_df, const_inputs=const_inputs):
    
    curr_epc_val = epc_data["EPC Cost"][0]
    
    new_val = curr_epc_val + value
    print(curr_epc_val, new_val)

    const_df_base = calc_epc_expenses(const_df, const_inputs, new_val=new_val)
    const_exp = make_const_exp(const_df_base) # return const_exp
    
    exp_df = calc_exp_df(const_exp, op_exp, corp_exp)

    _, _, roe, _ = run_model(ppa_df, exp_df, debt_service_df, dep_df)

    return roe

In [12]:
roes = {v:eval_epc_fluct(k) for k,v in zip(dev_cash, dev_cash) }
roes

455000000.0 371500000.0
455000000.0 379850000.0
455000000.0 388200000.0
455000000.0 396550000.0
455000000.0 404900000.0
455000000.0 413250000.0
455000000.0 421600000.0
455000000.0 429950000.0
455000000.0 438300000.0
455000000.0 446650000.0
455000000.0 455000000.0
455000000.0 463350000.0
455000000.0 471700000.0
455000000.0 480050000.0
455000000.0 488400000.0


{-83500000.0: -250.70983764837865,
 -75150000.0: -253.52861523016912,
 -66800000.0: -256.38281301055633,
 -58450000.0: -259.27243098954034,
 -50100000.0: -262.1974691671211,
 -41750000.0: -265.1579275432986,
 -33400000.0: -268.1538061180729,
 -25050000.0: -271.185104891444,
 -16700000.0: -274.25182386341186,
 -8350000.0: -277.35396303397647,
 0.0: -280.4915224031378,
 8350000.0: -283.66450197089597,
 16700000.0: -286.8729017372509,
 25050000.0: -290.1167217022026,
 33400000.0: -293.3959618657511}